In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec  1 15:34:54 2021

@author: p4u1
"""

'\nCreated on Wed Dec  1 15:34:54 2021\n\n@author: p4u1\n'

# Table of Contents

* [Optimization functions](#functions)
* [Testing](#test)
    * [Simplex](#simplex)
        * [Primal Form](#primal)
        * [Dual Form](#dual)
    * [Interior Point](#interiorPoint)
        * [Primal Form](#primal2)
        * [Dual Form](#dual2)
* [Error Messages](#error)

## Optimization functions <a class="anchor" id="functions"></a>

In [2]:
import numpy as np
import scipy.optimize as opt

In [3]:
def primalLinearProgOpt(A, meth = 'simplex', tol=1e-9):
    """ Wrapper function to get primal optimization results
        Uses scipy.optimize.linprog method
    
    Arguments:  A - square n x n payoff matrix (must be ndarray)
                n - size of matrix A
                method - linear programming method to use
                
    Returns:    Optimization variable values
                Objective function value"""

    # Objective function coefficents
    n = len(A)
    c0 = [0 for i in range(n)]
    c0.append(-1)
    c0 = np.array(c0)
    
    # Inequality contraints
    a = np.ones((n,1))
    A_u = np.concatenate((-(A.T), a), axis = 1)
    b_u = np.zeros(n)
    
    # Equality contraints
    A_e = [1 for i in range(n)]
    A_e.append(0)
    A_e = np.array(A_e)
    A_e = A_e.reshape((1,n+1))
    b_e = np.array(1)
    
    # Bounds
    bound = [(0, None) for i in range(n)]
    bound.append((None,None))
    
    # Run scipy.optimize.linprog method
    results = opt.linprog(c=c0,
                          A_ub=A_u, b_ub=b_u,
                          A_eq=A_e, b_eq=b_e,
                          bounds=bound, 
                          method=meth,
                          options = {'tol':tol})
    
    return results.x

# Test area
# A = np.array([[0,-1,1],[1,0,-1],[-1,1,0]])
# n = 3

# resultsPrimal = primalLinearProgOpt(A, n, 'interior-point')
# resultsDual = dualLinearProgOpt(A, n)

In [4]:
def dualLinearProgOpt(A, meth='simplex', tol=1e-9):
    """ Wrapper function to get dual optimization results
        Uses scipy.optimize.linprog method
    
    Arguments:  A - square n x n payoff matrix (must be ndarray)
                n - size of matrix A
                method - linear programming method to use
                
    Returns:    Optimization variable values"""

    # Objective function coefficents
    n = len(A)
    c0 = [0 for i in range(n)]
    c0.append(1)
    c0 = np.array(c0)
    
    # Inequality contraints
    a = np.ones((n,1))
    A_u = np.concatenate((A, -a), axis = 1)
    b_u = np.zeros(n)
    
    # Equality contraints
    A_e = [1 for i in range(n)]
    A_e.append(0)
    A_e = np.array(A_e)
    A_e = A_e.reshape((1,n+1))
    b_e = np.array(1)
    
    # Bounds
    bound = [(0, None) for i in range(n)]
    bound.append((None,None))
    
    # Run scipy.optimize.linprog method
    results = opt.linprog(c=c0,
                          A_ub=A_u, b_ub=b_u,
                          A_eq=A_e, b_eq=b_e,
                          bounds=bound, 
                          method=meth,
                          options={'tol':tol})
    
    return results.x

## Testing <a class="anchor" id="test"></a>

In [5]:
import time
%run util_matrix_generators.ipynb

### Simplex Method <a class="anchor" id="simplex"></a>

#### Primal Form <a class="anchor" id="primal"></a>

In [19]:
# varying size of R
times = []
for n in range(1, 300, 10):
    A = generate_R_uniform2(100, 200, n)
    start = time.time()
    primalLinearProgOpt(A)
    end = time.time()
    times.append(end - start)

c:\users\sshim\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:40: OptimizeWarning: The pivot operation produces a pivot value of: 7.8e-06, which is only slightly greater than the specified tolerance 1.0e-09. This may lead to issues regarding the numerical stability of the simplex method. Removing redundant constraints, changing the pivot strategy via Bland's rule or increasing the tolerance may help reduce the issue.
c:\users\sshim\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:40: OptimizeWarning: The pivot operation produces a pivot value of: 9.6e-06, which is only slightly greater than the specified tolerance 1.0e-09. This may lead to issues regarding the numerical stability of the simplex method. Removing redundant constraints, changing the pivot strategy via Bland's rule or increasing the tolerance may help reduce the issue.


In [17]:
times

[0.001962900161743164,
 0.024590015411376953,
 0.08903264999389648,
 0.1343059539794922,
 0.41256070137023926,
 0.4614086151123047,
 0.4819817543029785,
 0.5792291164398193,
 0.9858348369598389,
 0.5828876495361328,
 0.6237852573394775,
 0.6460206508636475,
 0.7052896022796631,
 0.773726224899292,
 0.9652533531188965,
 1.4024240970611572,
 0.8910093307495117,
 0.9560041427612305,
 1.0096931457519531,
 1.249009370803833]

#### Dual Form <a class="anchor" id="dual"></a>

### Interior Point Method <a class="anchor" id="interiorPoint"></a>

#### Primal Form <a class="anchor" id="primal2"></a>

In [12]:
# varying size of R
interior_primal_times = []
n = 1
while n < 1000:
    times = []
    for i in range(10):
        A = generate_R_uniform(-10, 10, n)
        start = time.time()
        primalLinearProgOpt(A, meth='interior-point')
        end = time.time()
        times.append(end-start)
    interior_primal_times.append((n, np.mean(times)))
    n *= 2

In [13]:
file = open('interior_primal_times.txt', 'w')
file.write(str(interior_primal_times))
file.close()

# file = open('interior_primal_times.txt', 'r')
# times2 = file.read()
# file.close

#### Dual Form <a class="anchor" id="dual2"></a>

In [14]:
# varying size of R
interior_dual_times = []
n = 1
while n < 1000:
    times = []
    for i in range(10):
        A = generate_R_uniform(-10, 10, n)
        start = time.time()
        dualLinearProgOpt(A, meth='interior-point')
        end = time.time()
        times.append(end-start)
    interior_dual_times.append((n, np.mean(times)))
    n *= 2

In [15]:
file = open('interior_dual_times.txt', 'w')
file.write(str(interior_dual_times))
file.close()

# file = open('interior_dual_times.txt', 'r')
# times2 = file.read()
# file.close

In [80]:
times2

'[(1, 0.0020101070404052734), (100, 0.017882823944091797), (200, 0.04781627655029297), (300, 0.08495903015136719), (400, 0.16666555404663086), (500, 0.2914552688598633), (600, 0.42180562019348145), (700, 0.7060284614562988), (800, 0.9646656513214111), (900, 1.209805965423584), (1000, 1.890902042388916), (1100, 2.2302403450012207), (1200, 3.093191623687744), (1300, 3.9851763248443604), (1400, 4.560537099838257), (1500, 6.072098970413208), (1600, 13.107242345809937), (1700, 15.38206958770752), (1800, 16.33038568496704), (1900, 20.210949659347534), (2000, 21.570293426513672), (2100, 26.979156017303467), (2200, 31.0743088722229), (2300, 33.86396765708923), (2400, 35.6103241443634), (2500, 41.12813663482666), (2600, 45.29458951950073), (2700, 50.71218752861023), (2800, 55.60111451148987), (2900, 62.488285779953), (3000, 71.43944907188416), (3100, 83.61240267753601), (3200, 84.91663265228271), (3300, 89.61718344688416), (3400, 97.50626873970032), (3500, 112.08613157272339), (3600, 119.244340

## Error Messages <a class="anchor" id="error"></a>

#### Simplex Method past 60x60 size R (both primal and dual), using generate_R_uniform(-1, 1, n)

Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 3.4e-01 which exceeds the required tolerance of 1e-09 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 3.4e-01. If this tolerance is unacceptably large the problem may be infeasible

In [18]:
generate_R_uniform2(100, 200, 5)

array([[   0.        , -156.21071933, -101.75566897,  134.40988703,
        -168.32691167],
       [ 156.21071933,    0.        , -113.53193802, -133.90547443,
         197.73066495],
       [ 101.75566897,  113.53193802,    0.        ,  157.29800504,
        -196.91065927],
       [-134.40988703,  133.90547443, -157.29800504,    0.        ,
        -182.39965829],
       [ 168.32691167, -197.73066495,  196.91065927,  182.39965829,
           0.        ]])